# Argos Search Results Notebook

## Installations

In [69]:
pip install fake-useragent

Note: you may need to restart the kernel to use updated packages.


In [70]:
import requests
import json
from fake_useragent import UserAgent

In [71]:
import pandas as pd
import numpy as np

In [72]:
pip install selectorlib

Note: you may need to restart the kernel to use updated packages.


In [73]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selectorlib import Extractor
import time

## Loading Pre-Documented Gender Stereotyped Items

In [74]:
stereo_toys = pd.read_csv('predoc_stereotyped_items.csv', delimiter=',')
stereo_toys

,BOY,GIRL,NEUTRAL
0,vehicle toys,doll,toy animals
1,sport,domestic toys,books
2,military toys,educational art,educational teaching
3,race cars,clothes,musical games
4,outer space toys,dollhouses,games
...,...,...,...
67,toy rocket,barbie furniture set,NaN
68,soccer ball,NaN,NaN
69,blue ipad,pink ipad,NaN
70,toy robots,NaN,NaN


In [75]:
with open('all_items.txt') as f:
    all_items = f.read().splitlines()

In [76]:
len(all_items)

166

In [77]:
trial = all_items[160:]
trial

['legos', 'scooter', 'drum set', 'puzzles', 'board games', 'rock painting']

In [78]:
generic = ['toys', 'books', 'learning material', 'games', 'sports']

In [79]:
gender = ['boys', 'girls', 'neutral']

## Search

### Collecting ASIN of Retrieved Products

In [109]:
def asin(driver):
    asin_list = []
    for index in range(1, 10):
        asins = driver.find_elements('xpath','//a[@aria-labelledby]') # is this going through each individual page?
        for asin in asins:
            time.sleep(0.5)
            asin_list.append(asin.get_attribute('aria-labelledby'))
    return asin_list

### Get Title Information of Retrieved Products

In [110]:
from selenium.webdriver.common.by import By
def item_info(driver):
    item = []
    elem = driver.find_elements('xpath', "//a[id/@data-test]")
    for i in elem:
        time.sleep(0.5)
        item.append(i.text)
    return item

### Collect Product Link of Retrieved Products

In [111]:
def item_link(driver):
    href = []
    #links = driver.find_elements('xpath', "//a[@data-test = 'component-product-card-link']")
    #for link in links;
        #time.sleep(0.5)
        #href.append(link.get_attribute('href'))
    return 0 #href

## Running Queries for Boys, Girls, and Neutral

In [112]:
def search(item, who):
    if who == 'neutral':
        query = item + '-for-' + 'kids'
    else:
        query = item + '-for-' + who
    driver.get(f'https://www.argos.co.uk/search/{query}/?clickOrigin=searchbar:home:term:{query}')
    list_asin = asin(driver)
    item_list = item_info(driver)
    item_page = item_link(driver)
    return (list_asin, item_list), item_page

In [113]:
columns1 = ['gender', 'query', 'result']
qr = pd.DataFrame(columns=columns1)
columns2 = ['gender', 'query', 'href']
qr_link = pd.DataFrame(columns=columns2)

In [114]:
trial

['legos']

In [115]:
trial = trial[0:1]

In [116]:
trial

['legos']

## Running Queries in a Loop

In [117]:
import warnings
warnings.filterwarnings('ignore')
driver = webdriver.Chrome(ChromeDriverManager().install())
data1 = []
data2 = []
item = ''
for item in trial:
    for g in gender:
        result, link = search(item, g)
        values1 = [g, item, result]
        values2 = [g, item, link]
        zipped1 = zip(columns1, values1)
        zipped2 = zip(columns2, values2)
        a_dictionary1 = dict(zipped1)
        a_dictionary2 = dict(zipped2)
        time.sleep(1.5)
        data1.append(a_dictionary1)
        data2.append(a_dictionary2)
driver.close()

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=110.0.5481.100)


In [ ]:
qr = qr.append(data1, True)
qr

In [ ]:
qr['result'][0]

In [ ]:
qr_link = qr_link.append(data2, True)
qr_link

In [ ]:
qr_link.href[1]

In [ ]:
first20 = pd.DataFrame()

In [ ]:
first20 = first20.append(qr, ignore_index = True)

In [ ]:
first20

In [ ]:
len(first20.loc[0]['result'][1])

In [ ]:
first20.to_csv('item_results.csv', index = False)